#AI - OCR

### Required package

In [ ]:
#!pip instal torch
!pip install EasyOCR
#!pip install opencv-python-headless==4.1.2.30
!pip install --ignore-installed PyYAML
#!pip install attrdict

In [2]:
# Getting the codes from github repositories.
!git clone https://github.com/aziz-ullah-khan/EasyOCR.git
!git clone https://github.com/Belval/TextRecognitionDataGenerator.git

Cloning into 'EasyOCR'...
remote: Enumerating objects: 2244, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 2244 (delta 0), reused 11 (delta 0), pack-reused 2229
Receiving objects: 100% (2244/2244), 163.37 MiB | 22.21 MiB/s, done.
Resolving deltas: 100% (1362/1362), done.
Checking out files: 100% (252/252), done.
Cloning into 'TextRecognitionDataGenerator'...
remote: Enumerating objects: 1175, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 1175 (delta 0), reused 1 (delta 0), pack-reused 1171
Receiving objects: 100% (1175/1175), 135.44 MiB | 17.51 MiB/s, done.
Resolving deltas: 100% (568/568), done.
Checking out files: 100% (275/275), done.


## Collecting/Generating Data

In [3]:
# Download data using google drive file id
!gdown --id 1Bz78_mC8IOVywJoGRWTGpCqzAaZqUNXV

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1Bz78_mC8IOVywJoGRWTGpCqzAaZqUNXV
To: /content/jp-dataset.zip
100% 11.5G/11.5G [00:50<00:00, 227MB/s]


In [4]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
# Unzip the file- Change the directory according to the training directory
!unzip "/content/jp-dataset.zip" -d '/content/'

Streaming output truncated to the last 5000 lines.
  inflating: /content/1moji-ground-truth/yoko021_20211203112913_00002_page_1_character_119_0.tif  
  inflating: /content/1moji-ground-truth/tate040_20211203122018_00003_page_2_character_76_0.tif  
  inflating: /content/1moji-ground-truth/yoko060_20211206162503_00003_page_1_character_31_0.gt.txt  
  inflating: /content/1moji-ground-truth/tate038_20211203120429_00006_page_2_character_65_0.tif  
  inflating: /content/1moji-ground-truth/tate070_20220111104112_00011_page_2_character_65_0.gt.txt  
  inflating: /content/1moji-ground-truth/tate041_20211203122333_00006_page_2_character_71_0.gt.txt  
  inflating: /content/1moji-ground-truth/yoko038_20211203120317_00001_page_4_character_34_0.tif  
  inflating: /content/1moji-ground-truth/yoko052_20211203125633_00002_page_3_character_14_0.gt.txt  
  inflating: /content/1moji-ground-truth/yoko064_20220107104240_00006_page_1_character_99_0.tif  
  inflating: /content/1moji-ground-truth/tate017_20211

In [ ]:
# Changing directory 
%cd /content/TextRecognitionDataGenerator/

# Install required packages
!pip install -r requirements.txt

%cd /content/TextRecognitionDataGenerator/trdg/

# create training samples with count = 100k with wrods = 5, -|---> change directory 
!python run.py -c 100000 -l ja --output_dir /content/genenerated_data/

In [ ]:
%cd /content/TextRecognitionDataGenerator

In [ ]:
# Generate data with manual custom data

from trdg.generators import (
    GeneratorFromDict,
    GeneratorFromRandom,
    GeneratorFromStrings,
    GeneratorFromWikipedia,
)
# ['渥'] contain the required samples, count is total number of samples to generate, lanauge ja = Japanese
generator = GeneratorFromStrings(
    ['A B 1 2 3 4 5 6 7 8 9', 'C D 5 6 7 8', 'T C I - USER technocrea.co.jp', '0 3 0 4 86 23482'],
    count = 500,
    language = 'ja',
)

# saving images
n = 0
for img, lbl in generator:  
  n+=1
  image_name = f"/content/genenerated_data/{lbl}_{n}.jpg"
  img.save(image_name)

### Importing required packages

In [ ]:
#change directory to import train file accessories
%cd /content/EasyOCR/trainer     

In [ ]:
import shutil
import os
import pandas as pd
import torch.backends.cudnn as cudnn
import yaml
from train import train
from utils import AttrDict
from sklearn.model_selection import train_test_split
import cv2
import numpy as np
import random
from google.colab.patches import cv2_imshow

### Preparing training Data

In [ ]:
# create directories for training and evaluation
%mkdir '/content/EasyOCR/trainer/all_data/ja_train_filtered/'
%mkdir '/content/EasyOCR/trainer/all_data/ja_val/'
%mkdir '/content/processed_images/'

In [ ]:
# initialize directories
BASE_DIR = '/content/1moji-ground-truth/'       # Base directory of the unzip folder
processed_images_dir = '/content/processed_images/'  # directory of final processed images
target_dir_training = '/content/EasyOCR/trainer/all_data/ja_train_filtered/'  
target_dir_val = '/content/EasyOCR/trainer/all_data/ja_val/'
genenerated_data_dir = '/content/genenerated_data'

In [ ]:
training_images_names = []     
training_text_data = []

training_files = os.listdir(BASE_DIR)

for training_file in training_files:
  if training_file.endswith(('.jpg', '.png', 'jpeg', 'tif')):
    image_name_no_extension = training_file.split('.')[0]
    training_text_file = image_name_no_extension + '.gt.txt'
    training_text_file_directory = BASE_DIR + training_text_file
   # print(training_text_file_directory)
    if os.path.isfile(training_text_file_directory):
      with open(training_text_file_directory) as f:
        training_images_names.append(training_file)
        training_text_data.append(f.read())
    else:
      pass
      #print ("text File does not exist!") 

In [ ]:
df_images_text_prepared = pd.DataFrame({'filename': training_images_names,
                  'words': training_text_data    
                   })
df_images_text_prepared.head()

In [ ]:
# merging images 
def combine_horizontally(image_names,padding, BASE_DIR):
    images = []
    max_height = 0  # find the max height of all the images
    total_width = 0  # the total width of the images (horizontal stacking)

    for name in image_names:
        # open all images and find their sizes
        img = cv2.imread(BASE_DIR + name)
        height = 32
        width = 32
        dim = (width, height)
        # resize image
        img  = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        images.append(img)

        image_height = img.shape[0]
        image_width = img.shape[1]


        if image_height > max_height:
            max_height = image_height

        # add all the images widths
        total_width += image_width

    # create a new array with a size large enough to contain all the images
    # also add padding size for all the images except the last one
    final_image = np.zeros((max_height, (len(image_names)-1)*padding+ total_width, 3), dtype=np.uint8)

    current_x = 0  # keep track of where your current image was last placed in the x coordinate
    for image in images:
        # add an image to the final array and increment the x coordinate
        height = image.shape[0]
        width = image.shape[1]
        final_image[:height,current_x :width+current_x, :] = image
        #add the padding between the images
        current_x += width+padding

    return final_image

In [ ]:
# random selection of images and text for combination from existing images, [3, 8, 10, 18, ....] 
random_selection = []
processed_images = []
processed_text = []
end_index = len(df_images_text_prepared.index)
idx = 0
start_index = 0
current_index = 0
for itrn in range(end_index):
  rand_idx = random.randint(1,15)
  idx +=rand_idx
  #print(idx)
  random_selection.append(idx)
  if idx > end_index or idx == end_index-1:
    break

for i, itr in enumerate(random_selection):
  if i == 0:
    start_index = 0
    current_index = itr+1
  else:
   # print(f"itrre {itr}")
    start_index = current_index
    current_index = itr+1

  df2 = df_images_text_prepared.iloc[start_index:current_index]
  final_image=combine_horizontally(df2['filename'].values.tolist(),0, BASE_DIR)
  final_text = [' '.join(df2['words'].values.tolist())]
  #cv2_imshow(final_image)
  img_name = processed_images_dir+final_text[0]+'.jpg'
  processed_images.append(img_name)
  processed_text.append(final_text)
  cv2.imwrite(img_name, final_image)

# create data frame
data = pd.DataFrame()
data['filename'] =  processed_images
data['words'] = processed_text


In [ ]:
# generate text from generated images and create df
training_files = os.listdir(genenerated_data_dir)
gen_data = pd.DataFrame(training_files, columns = ['filename'])
gen_data['words'] = gen_data['filename'].str.split(r'_\d+.jpg', expand = True)[0]

In [ ]:
# copy generated images to processed directory
source_dir = genenerated_data_dir
target_dir = processed_images_dir
for file in os.listdir(source_dir):
  shutil.copy2(os.path.join(source_dir, file), target_dir)

In [ ]:
#data = pd.DataFrame()
#data['filename'] = df['filename'].values.tolist() + processed_images
#data['words'] = df['words'].values.tolist() + processed_text

#data.head()

In [ ]:
# combine generated data and processed data
frames = [data, gen_data]
processed_data = pd.concat(frames)

Training and Validation Data Preparing

In [ ]:
# spliting data and copy totraining and val dir
training_data, validation_data = train_test_split(processed_data, test_size=0.2)

print('Total files: ', len(processed_data))
print('Total training files: ', len(training_data))
print('Total val files: ', len(validation_data))



for file in training_data['filename']:
  shutil.copy2(os.path.join(processed_images_dir, file), target_dir_training)

for file in validation_data['filename']:
  shutil.copy2(os.path.join(processed_images_dir, file), target_dir_val)

# Save the training data to csv
training_data.to_csv(target_dir_training + 'labels.csv', encoding = 'utf-8', index = False)
validation_data.to_csv(target_dir_val + 'labels.csv', encoding = 'utf-8', index = False)

## Training the Model

In [ ]:
cudnn.benchmark = True
cudnn.deterministic = False

In [ ]:
def get_config(file_path):
    with open(file_path, 'r', encoding="utf8") as stream:
        opt = yaml.safe_load(stream)
    opt = AttrDict(opt)
    if opt.lang_char == 'None':
        characters = ''
        for data in opt['select_data'].split('-'):
            csv_path = os.path.join(opt['train_data'], data, 'labels.csv')
            df = pd.read_csv(csv_path, sep=',', engine='python', usecols=['filename', 'words'], keep_default_na=False)
            all_char = ''.join(df['words'])
            characters += ''.join(set(all_char))
        characters = sorted(set(characters))
        opt.character= ''.join(characters)
    else:
        opt.character = opt.number + opt.symbol + opt.lang_char
    os.makedirs(f'./saved_models/{opt.experiment_name}', exist_ok=True)
    return opt


In [ ]:
opt = get_config("config_files/ja_filtered_config.yaml")
train(opt, amp=False)


## Loading the Trained Model

Copy custom_model.py custom_model.yaml  to the root directory: /root/EasyOCR/user_network/

copy custom_model.pth to /root/EasyOCR/model/


In [ ]:
!cp /root/EasyOCR/user_network/custom_model.yaml /root/.EasyOCR/user_network/custom_model.yaml
!cp /root/EasyOCR/user_network/custom_model.py /root/.EasyOCR/user_network/custom_model.py
!cp /root/EasyOCR/model/custom_model.pth /root/.EasyOCR/model/custom_model.pth

In [ ]:
import easyocr
reader = easyocr.Reader(['ja'], recog_network='custom_model')
result = reader.readtext('/content/EasyOCR/trainer/all_data/ja_val/tate001_20211203101858_00001_page_1_character_102_0.tif', detail = 0)  # remove detail =0 for details.. 

In [ ]:
result = reader.readtext('/content/name.jpg', detail = 0)  # remove detail =0 for details.. 
result

['獅', '秀', '腫', '腫', '物']

In [ ]:
!pip install tifffile 
!pip install imagecodecs
!pip install imagecodecs-lite